In [2]:
import tqdm.notebook as tqdm

import json
import os

from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from models import DisplayName

In [3]:
def plot_icon_on_map(map_image: Image.Image, x: float, y: float, icon: Image.Image):
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    coord_image_x = int((x - min_x) / (max_x - min_x) * image_width)
    coord_image_y = int((y - min_y) / (max_y - min_y) * image_height)

    icon_size = 80

    icon = icon.resize((icon_size, icon_size), Image.LANCZOS)
    icon_width, icon_height = icon.size

    icon_x = coord_image_x - icon_width // 2
    icon_y = coord_image_y - icon_height // 2

    map_image.paste(icon, (icon_x, icon_y), icon)

    return map_image

def plot_circle_on_map(map_image: Image.Image, x: float, y: float, radius: float, color: tuple):
    # draw a filled circle on map
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    coord_image_x = int((x - min_x) / (max_x - min_x) * image_width)
    coord_image_y = int((y - min_y) / (max_y - min_y) * image_height)

    draw = ImageDraw.Draw(map_image)

    circle_x0 = coord_image_x - radius
    circle_y0 = coord_image_y - radius
    circle_x1 = coord_image_x + radius
    circle_y1 = coord_image_y + radius

    draw.ellipse([circle_x0, circle_y0, circle_x1, circle_y1], fill=color)

    return map_image

def plot_text_on_map(map_image: Image.Image, x: float, y: float, text: str, font: ImageFont.FreeTypeFont, color: tuple, angle: float = 0):
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    text_bounding_box = font.getbbox(text)
    text_width = text_bounding_box[2] - text_bounding_box[0]
    text_height = text_bounding_box[3] - text_bounding_box[1]
    text_image = Image.new('RGBA', (text_width, text_height))
    text_draw = ImageDraw.Draw(text_image)
    text_draw.text((0, 0), text, font=font, fill=color, anchor='lt')
    text_image = text_image.rotate(angle, expand=1)

    coord_image_x = int((x - min_x) / (max_x - min_x) * image_width)
    coord_image_y = int((y - min_y) / (max_y - min_y) * image_height)

    text_x = coord_image_x - text_image.size[0] // 2
    text_y = coord_image_y - text_image.size[1] // 2 

    map_image.paste(text_image, (text_x, text_y), text_image)

    return map_image


def plot_box_on_map(map_image: Image.Image, start: tuple, end: tuple, color: tuple):
    # draw a filled box on map
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    start_image_x = int((start[0] - min_x) / (max_x - min_x) * image_width)
    start_image_y = int((start[1] - min_y) / (max_y - min_y) * image_height)

    end_image_x = int((end[0] - min_x) / (max_x - min_x) * image_width)
    end_image_y = int((end[1] - min_y) / (max_y - min_y) * image_height)

    # box_image = Image.new('RGBA', map_image.size, (0, 0, 0, 0))
    box_image = Image.new('RGBA', (end_image_x - start_image_x, end_image_y - start_image_y), (0, 0, 0, 0))
    draw = ImageDraw.Draw(box_image)

    # draw.rectangle([start_image_x, start_image_y, end_image_x, end_image_y], fill=color)
    draw.rectangle([0, 0, end_image_x - start_image_x, end_image_y - start_image_y], fill=color)
    
    map_image.paste(box_image, (start_image_x, start_image_y), box_image)

    return map_image

def plot_rounded_box_on_map(map_image: Image.Image, start: tuple, end: tuple, color: tuple, radius: int = 12):
    # draw a filled box on map
    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000

    image_width, image_height = map_image.size

    start_image_x = int((start[0] - min_x) / (max_x - min_x) * image_width)
    start_image_y = int((start[1] - min_y) / (max_y - min_y) * image_height)

    end_image_x = int((end[0] - min_x) / (max_x - min_x) * image_width)
    end_image_y = int((end[1] - min_y) / (max_y - min_y) * image_height)

    # box_image = Image.new('RGBA', map_image.size, (0, 0, 0, 0))
    box_image = Image.new('RGBA', (end_image_x - start_image_x, end_image_y - start_image_y), (0, 0, 0, 0))
    draw = ImageDraw.Draw(box_image)

    # draw.rectangle([start_image_x, start_image_y, end_image_x, end_image_y], fill=color)
    draw.rounded_rectangle([0, 0, end_image_x - start_image_x, end_image_y - start_image_y], fill=color, radius=radius)
    
    map_image.paste(box_image, (start_image_x, start_image_y), box_image)

    return map_image

def recolor_image(image: Image.Image, color: tuple):
    data = image.getdata()

    new_data = []
    for item in data:
        if item[0] >= 200 and item[1] >= 200 and item[2] >= 200:
            new_data.append((color[0], color[1], color[2], item[3]))
        else:
            new_data.append(item)

    image.putdata(new_data)

    return image

def rotate_image(image: Image.Image, angle: float):
    return image.rotate(angle, resample=Image.BICUBIC, expand=True)

In [3]:
def generate_normal_molars_map(molars_path: Path, map_path: Path):
    world_path = molars_path
    level_paths = {}
    for level_dir in world_path.iterdir():
        if not level_dir.is_dir():
            # print(f'{level_dir} is not a directory')
            continue

        level_paths[level_dir.name] = []
        for file in level_dir.iterdir():
            # if not file.name.endswith('Design_Actors.json'):
            #     # print(f'{file} is not a Design_Actors file')
            #     continue

            level_paths[level_dir.name].append(file)

    level_jsons = {}
    for level_id, locations in level_paths.items():
        level_jsons[level_id] = {}

        for location in locations:
            with open(location, 'r', encoding='utf-8') as f:
                level_jsons[level_id][location.stem] = json.load(f)

    # level_jsons['AR_00_World']['AR_00_World_01_01_Design_Actors']

    milk_molars = {}
    for level_id, locations in level_jsons.items():
        if len(locations) == 0:
            continue

        for location, components in locations.items():
            for index, component in enumerate(components):
                if 'Properties' not in component:
                    continue
                if 'RelativeLocation' not in component['Properties']:
                    continue
                if 'BP_Milk_Nugget' not in component['Outer'] and 'UpgradeCollectible' not in component['Outer']:
                    continue

                molar_type = ''
                if 'Personal' in component['Outer'] or component['Outer'].startswith('BP_UpgradeCollectible'):
                    molar_type = 'white'
                elif 'Party' in component['Outer'] or component['Outer'].startswith('BP_PartyUpgradeCollectible'):
                    molar_type = 'gold'

                underwater = False
                if 'UpgradeCollectible' in component['Outer']:
                    underwater = True

                component_location = component['Properties']['RelativeLocation']
                coords = (component_location['X'], component_location['Y'], component_location['Z'])
                milk_molars[f'{location}.{component['Outer']}'] = {
                    'coords': coords,
                    'type': molar_type,
                    'underwater': underwater,
                    'outer': component['Outer']
                }

    map_image = Image.open(map_path).convert('RGBA')
    map_image = map_image.resize((11264, 11264), Image.NEAREST)

    white_map_icon = Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/UI/Images/System/T_UI_MM_Tooth.png').convert('RGBA')
    yellow_map_icon = white_map_icon.copy()
    yellow_map_icon = recolor_image(yellow_map_icon, (235, 208, 0))

    white_map_icon = rotate_image(white_map_icon, -90)
    yellow_map_icon = rotate_image(yellow_map_icon, -90)

    for index, milk_molar in enumerate(milk_molars.values()):
        x, y, z = milk_molar['coords']
        molar_type = milk_molar['type']

        icon = white_map_icon if molar_type == 'white' else yellow_map_icon

        map_image = plot_icon_on_map(map_image, x, y, icon)
        # map_image = plot_text_on_map(map_image, x, y, str(index + 1), ImageFont.truetype('GillSansMTPro-Condensed.ttf', 20), (0, 0, 0), -90)
        # map_image = plot_text_on_map(map_image, x, y, milk_molar['outer'], ImageFont.truetype('GillSansMTPro-Condensed.ttf', 20), (0, 0, 0), -90)

    map_image = rotate_image(map_image, 90)
    map_image.save('milk_molars.png')

In [8]:
def generate_ngplus_molars_map(molars_path: Path, map_path: Path):
    world_00_path = Path(molars_path)
    world_00_json = json.load(open(world_00_path, 'r', encoding='utf-8'))

    # The following definition was extracted by looking manually at the json file
    world_00_json = world_00_json[70]['Properties']['ItemSpawnGroups']
    world_00_json = [
        world_00_json[20]["SG_NG+_MilkMolars_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/SG_NG+_MilkMolars.Default__SG_NG+_MilkMolars_C'"],
        world_00_json[21]["SG_NG+_MilkMolarsUnderwater_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/SG_NG+_MilkMolarsUnderwater.Default__SG_NG+_MilkMolarsUnderwater_C'"],
    ]
    world_00_json = {
        'normal': world_00_json[0]['SpawnPointDatas'],
        'underwater': world_00_json[1]['SpawnPointDatas']
    }

    milk_molars_ngplus = []
    for molar in world_00_json['normal']:
        coords = molar['SpawnTransform']['Translation']
        milk_molars_ngplus.append({
            'coords': (coords['X'], coords['Y'], coords['Z']),
            'type': 'white',
            'underwater': False
        })

    for molar in world_00_json['underwater']:
        coords = molar['SpawnTransform']['Translation']
        milk_molars_ngplus.append({
            'coords': (coords['X'], coords['Y'], coords['Z']),
            'type': 'gold',
            'underwater': True
        })

    print(len(milk_molars_ngplus))

    map_image = Image.open(map_path).convert('RGBA')
    map_image = map_image.resize((10000, 10000), Image.NEAREST)

    white_map_icon = Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/media_data/Maine/Content/UI/Images/System/T_UI_MM_Tooth.png').convert('RGBA')
    white_map_icon = rotate_image(white_map_icon, -90)

    # set the blue to be a water blue
    blue_map_icon = recolor_image(white_map_icon.copy(), (0, 84, 147))

    for index, milk_molar in enumerate(milk_molars_ngplus):
        x, y, z = milk_molar['coords']
        molar_type = milk_molar['type']
        underwater = milk_molar['underwater']

        icon = blue_map_icon if underwater else white_map_icon
        font_color = (255, 255, 255) if underwater else (0, 0, 0)

        map_image = plot_icon_on_map(map_image, x, y, icon)
        # map_image = plot_text_on_map(map_image, x, y, str(index + 1), ImageFont.truetype('GillSansMTPro-Condensed.ttf', 20), font_color, -90)
        map_image = plot_text_on_map(map_image, x, y, str(index + 1), ImageFont.truetype('WorkSans-Regular.ttf', 20), font_color, -90)

    map_image = rotate_image(map_image, 90)
    map_image.save('milk_molars_ngplus.png')

    txt_str = ''
    for index, milk_molar in enumerate(milk_molars_ngplus):
        x, y, z = milk_molar['coords']
        x = round(x, 2)
        y = round(y, 2)
        z = round(z, 2)
        molar_type = 'underwater' if milk_molar['underwater'] else 'normal'

        txt_str += f'{str(index + 1).rjust(3)} -> X: {str(x).ljust(10)} Y: {str(y).ljust(10)} Z: {str(z).ljust(10)} Type: {molar_type}\n'

    with open('milk_molars_ngplus.txt', 'w', encoding='utf-8') as f:
        f.write(txt_str)

In [5]:
def generate_candy_map(spawn_data_path: Path, map_path: Path):
    spawn_data_json = json.load(open(spawn_data_path, 'r', encoding='utf-8'))

    spawn_data_json = spawn_data_json[70]['Properties']['ItemSpawnGroups']
    spawn_data_json = {
        'rocket_mint': spawn_data_json[3]["SG_CandyRocket_Mint_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyRocket_Mint.Default__SG_CandyRocket_Mint_C'"]['SpawnPointDatas'],
        'rocket_sour': spawn_data_json[4]["SG_CandyRocket_Sour_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyRocket_Sour.Default__SG_CandyRocket_Sour_C'"]['SpawnPointDatas'],
        'rocket_spicy': spawn_data_json[5]["SG_CandyRocket_Spicy_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyRocket_Spicy.Default__SG_CandyRocket_Spicy_C'"]['SpawnPointDatas'],
        'box_mint': spawn_data_json[7]["SG_CandyBox_Mint_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyBox_Mint.Default__SG_CandyBox_Mint_C'"]['SpawnPointDatas'],
        'box_sour': spawn_data_json[8]["SG_CandyBox_Sour_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyBox_Sour.Default__SG_CandyBox_Sour_C'"]['SpawnPointDatas'],
        'box_spicy': spawn_data_json[9]["SG_CandyBox_Spicy_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyBox_Spicy.Default__SG_CandyBox_Spicy_C'"]['SpawnPointDatas'],
        'dispenser_mint': spawn_data_json[15]["SG_CandyDispenser_Mint_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyDispenser_Mint.Default__SG_CandyDispenser_Mint_C'"]['SpawnPointDatas'],
        'dispenser_sour': spawn_data_json[16]["SG_CandyDispenser_Sour_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyDispenser_Sour.Default__SG_CandyDispenser_Sour_C'"]['SpawnPointDatas'],
        'dispenser_spicy': spawn_data_json[17]["SG_CandyDispenser_Spicy_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/CandySpawnGroups/SG_CandyDispenser_Spicy.Default__SG_CandyDispenser_Spicy_C'"]['SpawnPointDatas'],
    }

    spawn_data = []
    for key, value in spawn_data_json.items():
        for item in value:
            coords = item['SpawnTransform']['Translation']
            coords = (coords['X'], coords['Y'], coords['Z'])

            spawn_data.append({
                'type': key,
                'coords': coords
            })

    map_image = Image.open(map_path).convert('RGBA')
    map_image = map_image.resize((4096, 4096), Image.NEAREST)

    icons = {
        'rocket_mint': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Wormholes_Container_A.png').convert('RGBA'),
        'rocket_sour': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Wormholes_Container_A.png').convert('RGBA'),
        'rocket_spicy': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Wormholes_Container_A.png').convert('RGBA'),
        'box_mint': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Hot_ChaCha_Box_A.png').convert('RGBA'),   
        'box_sour': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Hot_ChaCha_Box_A.png').convert('RGBA'),
        'box_spicy': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Hot_ChaCha_Box_A.png').convert('RGBA'),
        'dispenser_mint': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Ice_Cap_Mint_Box.png').convert('RGBA'),
        'dispenser_sour': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Ice_Cap_Mint_Box.png').convert('RGBA'),
        'dispenser_spicy': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Ice_Cap_Mint_Box.png').convert('RGBA'),
    }
    for key, value in icons.items():
        icons[key] = rotate_image(value, -90)

    def has_similar_coords(spawn_data: list, spawn_a: tuple):
        for spawn_b in spawn_data:
            diff_x = abs(spawn_b['coords'][0] - spawn_a['coords'][0])
            diff_y = abs(spawn_b['coords'][1] - spawn_a['coords'][1])
            spawn_type_a = spawn_a['type'].split('_')[0]
            spawn_type_b = spawn_b['type'].split('_')[0]
            if (diff_x <= 1000 and diff_y <= 1000) and (spawn_type_a == spawn_type_b):
                return True

        return False

    plotted_spawns = []
    for index, spawn in enumerate(spawn_data):
        x, y, z = spawn['coords']
        icon = icons[spawn['type']]

        if has_similar_coords(plotted_spawns, spawn):
            continue

        map_image = plot_icon_on_map(map_image, x, y, icon)
        # map_image = plot_text_on_map(map_image, x, y, f'{index}', 40, (255, 255, 255))

        plotted_spawns.append(spawn)

        # if index >= 3:
        #     break

    map_image = rotate_image(map_image, 90)
    map_image.save('candy_spawns.png')

In [6]:
def generate_ngplus_raw_science_map(spawn_data_path: Path, map_path: Path):
    spawn_data_json = json.load(open(spawn_data_path, 'r', encoding='utf-8'))

    spawn_data_json = spawn_data_json[70]['Properties']['ItemSpawnGroups']
    spawn_data_json = {
        'raw_science': spawn_data_json[3]["SG_NG+_Science_C'/Game/Blueprints/Items/SpawnPoints/SpawnGroups/SG_NG+_Science.Default__SG_NG+_Science_C'"]['SpawnPointDatas'],
    }

    spawn_data = []
    for key, value in spawn_data_json.items():
        for item in value:
            coords = item['SpawnTransform']['Translation']
            coords = (coords['X'], coords['Y'], coords['Z'])

            spawn_data.append({
                'type': key,
                'coords': coords
            })

    map_image = Image.open(map_path).convert('RGBA')
    map_image = map_image.resize((4096, 4096), Image.NEAREST)

    icons = {
        'raw_science': Image.open('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/media_data/Maine/Content/Blueprints/Items/Icons/ICO_Wormholes_Container_A.png').convert('RGBA'),
    }
    for key, value in icons.items():
        icons[key] = rotate_image(value, -90)

    def has_similar_coords(spawn_data: list, spawn_a: tuple):
        for spawn_b in spawn_data:
            diff_x = abs(spawn_b['coords'][0] - spawn_a['coords'][0])
            diff_y = abs(spawn_b['coords'][1] - spawn_a['coords'][1])
            spawn_type_a = spawn_a['type'].split('_')[0]
            spawn_type_b = spawn_b['type'].split('_')[0]
            if (diff_x <= 1000 and diff_y <= 1000) and (spawn_type_a == spawn_type_b):
                return True

        return False

    plotted_spawns = []
    for index, spawn in enumerate(spawn_data):
        x, y, z = spawn['coords']
        icon = icons[spawn['type']]

        if has_similar_coords(plotted_spawns, spawn):
            continue

        map_image = plot_icon_on_map(map_image, x, y, icon)
        # map_image = plot_text_on_map(map_image, x, y, f'{index}', 40, (255, 255, 255))

        plotted_spawns.append(spawn)

        # if index >= 3:
        #     break

    map_image = rotate_image(map_image, 90)
    map_image.save('candy_spawns.png')

In [87]:
molars = generate_normal_molars_map(
    molars_path=Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/json_data/Maine/Content/Levels/AR_00_World'),
    map_path=Path('D:/Backup/samjv/Desktop/code/python/grounded/combined_resized_image.png')
)

In [9]:
generate_ngplus_molars_map(
    molars_path=Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine/Content/Levels/AR_00_World/AR_00_World_Rel.json'),
    map_path=Path('D:/Backup/samjv/Desktop/code/python/grounded/combined_resized_image.png')
)

219


In [ ]:
generate_candy_map(
    spawn_data_path=Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/json_data/Maine/Content/Levels/AR_00_World/AR_00_World_Rel.json'),
    map_path=Path('D:/Backup/samjv/Desktop/code/python/grounded/combined_resized_image.png')
)

In [10]:
world_path = Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine/Content/Levels/AR_00_World')

level_paths: dict[str, list[Path]] = {}
world_prefix = world_path.name.split(f'_{world_path.name.split('_')[-1]}')[0]
for level_dir in world_path.iterdir():
    if level_dir.is_file() or (level_dir.is_file() and not level_dir.name.startswith(world_prefix)):
        # print(f'File: {level_dir}')
        continue
    elif not level_dir.is_dir():
        continue

    level_paths[level_dir.name] = [ file for file in level_dir.iterdir() ]

In [12]:
def get_item_info(item_id: str) -> dict:
    global all_items_datatable, loaded_items

    if item_id == 'None':
        return None

    if all_items_datatable is None:
        path = Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine/Content/Blueprints/Items/Table_AllItems.json')
        all_items_datatable = json.load(open(path, 'r', encoding='utf-8'))[0]['Rows']
    if item_id in loaded_items:
        return loaded_items[item_id]
    
    item_json = all_items_datatable[item_id]
    name = DisplayName(
        item_json['LocalizedDisplayName']['StringTableID'],
        item_json['LocalizedDisplayName']['StringID'],
        item_json['LocalizedDisplayName']['StringTableName']
    )
    description = DisplayName(
        item_json['LocalizedDescription']['StringTableID'],
        item_json['LocalizedDescription']['StringID'],
        item_json['LocalizedDescription']['StringTableName']
    )
    icon = item_json['Icon']['AssetPathName'].split('.')[0].replace('/Game/', '/Content/')
    icon = Path(f'C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/media_data/Maine{icon}.png')

    item_info = {
        'name': name,
        'description': description,
        'icon': icon
    }

    loaded_items[item_id] = item_info

    return item_info

def get_loot_info(object_path: str) -> dict:
    global global_foliage_data, loaded_loot

    if global_foliage_data is None:
        path = Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine/Content/Blueprints/Global/GlobalFoliageData.json')
        global_foliage_data = json.load(open(path, 'r', encoding='utf-8'))[0]['Properties']

    foliage_mappings = global_foliage_data['FoliageMappings']
    blueprint_object = None
    for foliage_mapping in foliage_mappings:
        key = list(foliage_mapping.keys())[0]
        if key == object_path:
            blueprint_object = foliage_mapping[key]['FoliageBlueprintClass']
            break

    if blueprint_object is None:
        return None

    blueprint_name = blueprint_object['ObjectPath'].split('/')[-1].split('.')[0]
    if blueprint_name in loaded_loot:
        return loaded_loot[blueprint_name]

    blueprint_path = blueprint_object['ObjectPath'].split('.')[0].replace('/Game/', '/Content/')
    blueprint_path = Path(f'C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine{blueprint_path}.json')
    with open(blueprint_path, 'r', encoding='utf-8') as f:
        blueprint_json = json.load(f)

    loot = get_loot_from_bp(blueprint_json)
    loaded_loot[blueprint_name] = loot

    return loot

def get_loot_from_bp(bp_json: dict) -> list[dict]:
    loot = []
    for component in bp_json:
        if component['Type'] != 'LootComponent':
            continue

        loot_table = component['Properties']['Items']
        for loot_item in loot_table:
            item_path = loot_item['ItemData']['DataTable']['ObjectPath'].split('.')[0].replace('/Game/', '/Content/')
            item_path = Path(f'C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine{item_path}.json')

            item_id = loot_item['ItemData']['RowName']

            loot.append({
                'item_path': item_path,
                'item_id': item_id,
                'item': get_item_info(item_id),
                'quantity': loot_item['Count'],
                'chance': loot_item['DropChance'],
                'ignore_luck': loot_item['bIgnoreLuck'],
                'stealable': loot_item['bStealable'],
                'ng+_tier': loot_item['RequiredNewGamePlusTier']
            })

    return loot

def get_bp_from_sd(sd_path: Path) -> list[dict]:
    with open(sd_path, 'r', encoding='utf-8') as f:
        sd_json = json.load(f)

    jsons = []
    for component in sd_json:
        if 'Properties' not in component or 'BlueprintPool' not in component['Properties']:
            continue

        # if len(component['Properties']['BlueprintPool']) > 1:
        #     print(f'Multiple pools in {sd_path}')
        
        bp_path = component['Properties']['BlueprintPool'][0]['AssetPathName'].split('.')[0].replace('/Game/', '/Content/')
        bp_path = Path(f'C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine{bp_path}.json')
        jsons.append(json.load(open(bp_path, 'r', encoding='utf-8'))) 

    return jsons

In [13]:
global_foliage_data = None
all_items_datatable = None
loaded_loot = {}
loaded_items = {}

DisplayName.init(Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine/Content/Exported/BaseGame/Localized/enus/Text/Text_enus.json'))

In [14]:
map_components = []

for level_id, level_files in level_paths.items():
    for level_file in level_files:
        with open(level_file, 'r', encoding='utf-8') as f:
            level_json = json.load(f)

            for component in level_json:
                if 'Outer' not in component:
                    continue
                if component['Outer'] == 'Encounter_RolyPoly_Underdeck':
                    print(component)
                if 'Properties' not in component:
                    continue

                if 'RelativeLocation' in component['Properties'] or 'Location' in component['Properties']:
                    location_str = 'Location' if 'Location' in component['Properties'] else 'RelativeLocation'

                    component_location = component['Properties'][location_str]
                    coords = (component_location['X'], component_location['Y'], component_location['Z'])

                    name = component['Outer']
                    # if 'lint' not in name.lower() or 'flint' in name.lower():
                    #     continue

                    map_components.append({
                        'coords': coords,
                        'name': name,
                        'items': [],
                        'key_name': component['Name']
                    })
                elif 'BuiltInstanceBounds' in component['Properties']:
                    bounds = component['Properties']['BuiltInstanceBounds']
                    
                    min_coords = (
                        bounds['Min']['X'],
                        bounds['Min']['Y'],
                        bounds['Min']['Z']
                    )
                    max_coords = (
                        bounds['Max']['X'],
                        bounds['Max']['Y'],
                        bounds['Max']['Z']
                    )
                    coords = (
                        (min_coords[0] + max_coords[0]) / 2,
                        (min_coords[1] + max_coords[1]) / 2,
                        (min_coords[2] + max_coords[2]) / 2
                    )

                    name = component['Properties']['StaticMesh']['ObjectPath'].split('/')[-1].split('.')[0]
                    # if 'lint' not in name.lower() or 'flint' in name.lower() or 'splinter' in name.lower():
                    #     continue

                    loot = get_loot_info(f'{component['Properties']['StaticMesh']['ObjectPath'].split('.')[0]}.{name}')
                    if loot is None:
                        loot = []

                    radius = component['Properties']['CacheMeshExtendedBounds']['SphereRadius']

                    box_bounds = component['Properties']['CacheMeshExtendedBounds']
                    if 'Origin' not in box_bounds:
                        box_bounds['Origin'] = {
                            'X': 0,
                            'Y': 0,
                            'Z': 0
                        }
                    origin = (
                        box_bounds['Origin']['X'],
                        box_bounds['Origin']['Y'],
                        box_bounds['Origin']['Z']
                    )
                    extent = (
                        box_bounds['BoxExtent']['X'],
                        box_bounds['BoxExtent']['Y'],
                        box_bounds['BoxExtent']['Z']
                    )
                    box = {
                        'origin': min_coords,
                        'extent': max_coords
                    }

                    map_components.append({
                        'name': name,
                        'items': loot,
                        'coords': coords,
                        'box': box,
                        'radius': radius
                    })

{'Type': 'EncounterComponent', 'Name': 'RootComponent', 'Outer': 'Encounter_RolyPoly_Underdeck', 'Class': "UScriptClass'EncounterComponent'", 'Properties': {'RelativeLocation': {'X': 70781.24, 'Y': -31086.213, 'Z': 4830.6055}}}
{'Type': 'ObsidianIDComponent', 'Name': 'ObsidianIDComponent', 'Outer': 'Encounter_RolyPoly_Underdeck', 'Class': "UScriptClass'ObsidianIDComponent'", 'Properties': {'ID': '61301D7F-479B0954-DB6A92B0-E1FA3DF3'}}
{'Type': 'PersistenceComponent', 'Name': 'PersistenceComponent', 'Outer': 'Encounter_RolyPoly_Underdeck', 'Class': "UScriptClass'PersistenceComponent'"}


In [15]:
spawn_groups_path = Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine/Content/Levels/AR_00_World/AR_00_World_Rel.json')
spawn_groups_json = json.load(open(spawn_groups_path, 'r', encoding='utf-8'))

In [16]:
for spawn_groups in spawn_groups_json:
    if 'Properties' not in spawn_groups or 'ItemSpawnGroups' not in spawn_groups['Properties']:
        continue
        
    spawn_groups = spawn_groups['Properties']['ItemSpawnGroups']
    for spawn_group in spawn_groups:
        spawn_group_id = list(spawn_group.keys())[0]
        spawn_group = spawn_group[spawn_group_id]

        if 'SpawnPointDatas' not in spawn_group:
            continue

        spawn_points = spawn_group['SpawnPointDatas']
        for spawn_point in spawn_points:
            coords = spawn_point['SpawnTransform']['Translation']
            coords = (coords['X'], coords['Y'], coords['Z'])

            name = spawn_point['ItemSpawnData']['ObjectPath'].split('/')[-1].split('.')[0]
            items = []
            if name.startswith('SD_'):
                sd_path = spawn_point['ItemSpawnData']['ObjectPath'].split('.')[0].replace('/Game/', '/Content/')
                sd_path = Path(f'C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4.0.4495/json_data/Maine{sd_path}.json')
                bp_jsons = get_bp_from_sd(sd_path)
                for bp_json in bp_jsons:
                    loot = get_loot_from_bp(bp_json)
                    items.extend(loot)

            map_components.append({
                'coords': coords,
                'name': name,
                'items': items
            })


In [ ]:
# Location or RelativeLocation
location_str = 'RelativeLocation'
map_components = []
for actor in actors_json:
    if 'Outer' not in actor or 'Properties' not in actor or location_str not in actor['Properties']:
        continue
    
    coords = (
        actor['Properties'][location_str]['X'],
        actor['Properties'][location_str]['Y'],
        actor['Properties'][location_str]['Z']
    )
    map_components.append({
        'name': actor['Outer'],
        'coords': coords
    })

In [ ]:
# foods
spawn_group_list = actors_json[70]['Properties']['ItemSpawnGroups']

spawn_groups = {}
for actor in spawn_group_list:
    key = list(actor.keys())[0]
    spawn_groups[key] = actor[key]

map_components = []
for spawn_group, spawn_datas in spawn_groups.items():
    if 'Food_Manmade' not in spawn_group:
        continue

    for spawn_data in spawn_datas['SpawnPointDatas']:
        coords = (
            spawn_data['SpawnTransform']['Translation']['X'],
            spawn_data['SpawnTransform']['Translation']['Y'],
            spawn_data['SpawnTransform']['Translation']['Z']
        )
        spawn_data_name = spawn_data['ItemSpawnData']['ObjectPath'].split('/')[-1].split('.')[0]
        name = f'{spawn_group.split('.')[-1]}.{spawn_data_name}'

        map_components.append({
            'name': spawn_group,
            'coords': coords
        })

In [ ]:
# spwan area
map_components = []
for actor in actors_json:
    if 'Outer' not in actor or 'Properties' not in actor or 'BuiltInstanceBounds' not in actor['Properties']:
        continue

    bounds = actor['Properties']['BuiltInstanceBounds']

    min_coords = (
        bounds['Min']['X'],
        bounds['Min']['Y'],
        bounds['Min']['Z']
    )
    max_coords = (
        bounds['Max']['X'],
        bounds['Max']['Y'],
        bounds['Max']['Z']
    )
    coords = (
        (min_coords[0] + max_coords[0]) / 2,
        (min_coords[1] + max_coords[1]) / 2,
        (min_coords[2] + max_coords[2]) / 2
    )

    name = actor['Properties']['StaticMesh']['ObjectPath'].split('/')[-1].split('.')[0]
    if 'lint' not in name.lower():
        continue

    radius = actor['Properties']['CacheMeshExtendedBounds']['SphereRadius']

    map_components.append({
        'name': name,
        'coords': coords,
        'radius': radius
    })

In [17]:
preloaded_icons = {}
for component in tqdm.tqdm(map_components):
    for item in component['items']:
        item_id = item['item_id']
        if item_id == 'None':
            continue
        if not os.path.exists(item['item']['icon']):
            continue
        if item_id in preloaded_icons:
            continue
    
        icon = Image.open(item['item']['icon']).convert('RGBA')
        icon = rotate_image(icon, -90)
        preloaded_icons[item_id] = icon

  0%|          | 0/35809 [00:00<?, ?it/s]

In [19]:
def filter_by_item(item_name: str, component: dict) -> bool:
    for loot in component['items']:
        if loot['item'] is None:
            continue

        if item_name.lower() in loot['item']['name'].text.lower():
            return True
        
    return False

map_image = Image.open(Path('D:/Backup/samjv/Desktop/code/python/grounded/combined_resized_image.png')).convert('RGBA')
map_image = map_image.resize((10000, 10000), Image.NEAREST)

scale = 100000 / 100000

item_name = 'rolypoly'
for component in tqdm.tqdm(map_components):
    if item_name.lower() not in json.dumps(component, default=str).lower() or 'trap' in json.dumps(component, default=str).lower() or 'decal' in json.dumps(component, default=str).lower():
        continue
    if component['name'] not in ['RolyPolyEncounter', 'RolyPolyEncounter2', 'RolyPolyEncounter3', 'RolyPolyEncounter4']:
        continue
    # if not filter_by_item(item_name, component):
    #     continue

    min_x = -100000
    max_x = 100000
    min_y = -100000
    max_y = 100000
 
    x, y, z = component['coords']
    
    if 'box' in component:
        box = component['box']
        start = box['origin']
        end = box['extent']

        map_image = plot_rounded_box_on_map(map_image, box['origin'], box['extent'], (255, 0, 0, 64), radius=14)
        
        icon = None
        if len(component['items']) > 0 and component['items'][0]['item'] is not None:
            icon = preloaded_icons[component['items'][0]['item_id']]

        if icon is not None:
            map_image = plot_icon_on_map(map_image, x, y, icon)
      
    else:
        if len(component['items']) > 0 and component['items'][0]['item'] is not None:
            icon = preloaded_icons[component['items'][0]['item_id']]
            map_image = plot_icon_on_map(map_image, x, y, icon)
        else:
            radius = 16
            if 'radius' in component:
                radius = component['radius']
                radius *= scale

            map_image = plot_circle_on_map(map_image, x, y, radius, (255, 0, 0))
        
    name = component['name']
    if len(component['items']) > 0 and component['items'][0]['item'] is not None:
        name = component['items'][0]['item']['name'].text

    map_image = plot_text_on_map(map_image, x, y, name, ImageFont.truetype('Gin.ttf', 32), (255, 255, 255), -90)

map_image = rotate_image(map_image, 90)
map_image.save('map_components.png')

  0%|          | 0/35809 [00:00<?, ?it/s]

In [ ]:
path = Path('C:/Users/PC-SAMUEL/Desktop/data_mining/grounded/1.4/json_data/Maine/Content/Levels/AR_00_World/AR_12_Sandbox/AR_12_Design_Actors_Caves.json')
data = json.load(open(path, 'r', encoding='utf-8'))

In [112]:
def calcular_probabilidade_completa(chance_roubar_skill, chance_roubar_accessory, chance_sortear, num_ataques):
    # Convertendo as porcentagens para forma decimal
    prob_roubar_skill = chance_roubar_skill / 100
    prob_roubar_accessory = chance_roubar_accessory / 100
    prob_sortear = chance_sortear / 100
    
    # Calculando a probabilidade combinada de não roubar com a skill ou o accessory
    prob_nao_roubar_skill = 1 - prob_roubar_skill
    prob_nao_roubar_accessory = 1 - prob_roubar_accessory
    prob_nao_roubar_ambos = prob_nao_roubar_skill * prob_nao_roubar_accessory
    
    # Probabilidade de roubar pelo menos uma vez com a skill ou o accessory
    prob_roubar_ambos = 1 - prob_nao_roubar_ambos
    
    # Probabilidade de roubar o item específico em uma tentativa
    prob_roubar_especifico = prob_roubar_ambos * prob_sortear
    
    # Calculando a probabilidade de não roubar o item específico em todas as tentativas
    prob_nao_roubar_especifico_todos_ataques = (1 - prob_roubar_especifico) ** num_ataques
    
    # Probabilidade de roubar o item específico pelo menos uma vez
    prob_roubar_especifico_pelo_menos_uma = 1 - prob_nao_roubar_especifico_todos_ataques
    
    return prob_roubar_especifico_pelo_menos_uma

In [117]:
print(f'with trinket: {calcular_probabilidade_completa(10, 10, 20, 14) * 100:.2f}%')
print(f'with trinket: {calcular_probabilidade_completa(10, 0, 20, 32) * 100:.2f}%')

with trinket: 41.86%
with trinket: 47.61%


In [171]:
import random

from IPython.display import display, HTML

In [175]:
def attack(health, damage):
    global counts
    health -= damage
    counts['attacks'] += 1
    return health

def steal(source, log):
    global valid_items, counts
    rand = random.randint(0, len(valid_items) - 1)
    if log:
        print(f'Steal: {valid_items[rand]} from {source}')

    counts[valid_items[rand]][source] += 1
    counts[valid_items[rand]]['total'] += 1
    counts['total'] += 1

attacks = [
    59 * 1.0 * 0.8 * 0.2,
    59 * 1.0 * 0.8 * 0.2,
    59 * 1.25 * 0.8 * 0.2
]
attacks = [
    59 * 1.0,
    59 * 1.0,
    59 * 1.25
]

steal_sources = [
    # 'rascal_rogue',
    'sticky_fingers',
]

valid_items = [
    'fungal_growth_1',
    'fungal_growth_2',
    'fungal_growth_3',
    'fungal_growth_4',
    'fungal_charm'        
]

def simulate(log=False):
    attack_count = 0
    health = 150
    combo = 0
    while health > 0:
        if log:
            print(f'Health: {health}')
        health = attack(health, attacks[combo])
        attack_count += 1
        combo += 1
        if combo >= len(attacks):
            combo = 0

        for source in steal_sources:
            rand = random.randint(1, 100)
            if rand <= 10:
                steal(source, log)
    if log:
        print(f'\nAttack count: {attack_count}')

In [176]:
# this will store information to calculate the probability of stealing an item
counts = {
    'fungal_growth_1': {
        'rascal_rogue': 0,
        'sticky_fingers': 0,
        'total': 0,
    },
    'fungal_growth_2': {
        'rascal_rogue': 0,
        'sticky_fingers': 0,
        'total': 0,
    },
    'fungal_growth_3': {
        'rascal_rogue': 0,
        'sticky_fingers': 0,
        'total': 0,
    },
    'fungal_growth_4': {
        'rascal_rogue': 0,
        'sticky_fingers': 0,
        'total': 0,
    },
    'fungal_charm': {
        'rascal_rogue': 0,
        'sticky_fingers': 0,
        'total': 0,
    },
    'total': 0,
    'attacks': 0
}

simulations = 1000000
for _ in tqdm.tqdm(range(simulations)):
    simulate()

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
print(json.dumps(counts, indent=4))

In [178]:
probabilities_html = '<table>'
probabilities_html += '<tr>'
probabilities_html += '<th>Item</th>'
probabilities_html += '<th>Rascal Rogue</th>'
probabilities_html += '<th>Sticky Fingers</th>'
probabilities_html += '<th>Total</th>'
probabilities_html += '</tr>'
for item in valid_items:
    probabilities_html += '<tr>'
    probabilities_html += f'<td>{item}</td>'
    probabilities_html += f'<td>{(counts[item]["rascal_rogue"] / counts[item]["total"]) * 100:.3f}</td>'
    probabilities_html += f'<td>{(counts[item]["sticky_fingers"] / counts[item]["total"]) * 100:.3f}</td>'
    probabilities_html += f'<td>{(counts[item]["total"] / counts["total"]) * 100:.3f}</td>'
    probabilities_html += '</tr>'
probabilities_html += '</table>'
probabilities_html += '<p>Chance of "fungal_charm": </p>'
probabilities_html += f'<p>{(counts["fungal_charm"]["total"] / counts["total"]) * 100:.3f}</p>'

display(HTML(probabilities_html))

Item,Rascal Rogue,Sticky Fingers,Total
fungal_growth_1,0.000,100.000,19.934
fungal_growth_2,0.000,100.000,20.084
fungal_growth_3,0.000,100.000,19.957
fungal_growth_4,0.000,100.000,19.857
fungal_charm,0.000,100.000,20.168


In [189]:
import random

def simulate_combat():
    health = 150
    attacks = [
        59 * 1.0 * 0.8 * 0.2,
        59 * 1.0 * 0.8 * 0.2,
        59 * 1.25 * 0.8 * 0.2
    ]
    items = ['fungal_growth_1', 'fungal_growth_2', 'fungal_growth_3', 'fungal_growth_4', 'fungal_charm']
    stolen_items = []
    attack_count = 0

    while health > 0:
        for attack in attacks:
            if health <= 0:
                break
            health -= attack
            # Check for item steal from Rascal Rogue
            if random.random() < 0.10:
                stolen_items.append(random.choice(items))
            # Check for item steal from Sticky Fingers
            if random.random() < 0.10:
                stolen_items.append(random.choice(items))

            attack_count += 1

    return stolen_items, attack_count

total_stolen_items = []
total_attack_count = 0
simulations = 100000
for _ in tqdm.tqdm(range(simulations)):
    stolen_items, attack_count = simulate_combat()
    total_stolen_items.extend(stolen_items)
    total_attack_count += attack_count
# stolen_items, attack_count = simulate_combat()
# print(f"Items stolen: {stolen_items} in {attack_count} attacks")

  0%|          | 0/100000 [00:00<?, ?it/s]

In [191]:
counts = {}
for item in total_stolen_items:
    if item not in counts:
        counts[item] = 0
    counts[item] += 1

In [199]:
probabilities_html = '<table>'
probabilities_html += '<tr>'
probabilities_html += '<th>Item</th>'
probabilities_html += '<th>Chance</th>'
probabilities_html += '</tr>'
for item in tqdm.tqdm(counts):
    probabilities_html += '<tr>'
    probabilities_html += f'<td>{item}</td>'
    probabilities_html += f'<td>{(counts[item] / len(total_stolen_items)) * 100:.3f}</td>'
    probabilities_html += '</tr>'
probabilities_html += '</table>'
probabilities_html += f'<p>Average attacks: {total_attack_count / simulations}</p>'
probabilities_html += f'<p>Chance per attack: {len(total_stolen_items) / total_attack_count * 100:.3f}</p>'
probabilities_html += f'<p>Chance of stealing fungal charm per attack: {counts["fungal_charm"] / total_attack_count * 100:.3f}</p>'
probabilities_html += f'<p>Chance of stealing fungal charm in a combo(3 attacks): {counts["fungal_charm"] / len(total_stolen_items) * 100:.3f}</p>'

display(HTML(probabilities_html))

  0%|          | 0/5 [00:00<?, ?it/s]

Item,Chance
fungal_growth_1,20.120
fungal_charm,19.871
fungal_growth_2,20.069
fungal_growth_4,19.777
fungal_growth_3,20.163
